# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

In [ ]:
from pyspark.sql import SparkSession
import os

data_dir = '../../data/Spark_ML/Tree_Methods/'

In [ ]:
spark = SparkSession.builder.appName('tree_consulting_project').getOrCreate()

data = spark.read.csv(os.path.join(data_dir, 'dog_food.csv'), inferSchema=True, header=True)

data.printSchema()

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')

output = assembler.transform(data)

final_data = output.select(['features', 'Spoiled'])

final_data.head(2)

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dtc = DecisionTreeClassifier(featuresCol='features', labelCol='Spoiled')

dtc_model = dtc.fit(final_data)

In [ ]:
dtc_preds = dtc_model.transform(final_data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

bin_eval = BinaryClassificationEvaluator(labelCol='Spoiled')

print('DTC: {}'.format(bin_eval.evaluate(dtc_preds)))

In [ ]:
## Feature importance

dtc_model.featureImportances

## 2nd feature, 'C' in that case is the most important feature.